In [1]:
import cv2,os

dpath='dataset'
categories=os.listdir(dpath)
labels=[i for i in range(len(categories))]
label_dict=dict(zip(categories,labels)) 
print(label_dict)
print(categories)
print(labels)

{'without_mask': 0, 'with_mask': 1}
['without_mask', 'with_mask']
[0, 1]


In [2]:
img_size=100
data=[]
tar=[]


for category in categories:
    folder_path=os.path.join(dpath,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            tar.append(label_dict[category])
            

        except Exception as e:
            print('Exception:',e)

In [3]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
tar=np.array(tar)

from keras.utils import np_utils

new_target=np_utils.to_categorical(tar)
np.save('data',data)
np.save('tar',new_target)

Using TensorFlow backend.


In [4]:
data=np.load('data.npy')
tar=np.load('tar.npy')

In [5]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(50,activation='relu'))

model.add(Dense(2,activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(data,tar,test_size=0.1)

In [7]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(X_train,y_train,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Train on 990 samples, validate on 248 samples
Epoch 1/20
990/990 [==============================] - 68s 69ms/step - loss: 0.6817 - accuracy: 0.5677 - val_loss: 0.6682 - val_accuracy: 0.5605
Epoch 2/20
990/990 [==============================] - 84s 85ms/step - loss: 0.4725 - accuracy: 0.7828 - val_loss: 0.4404 - val_accuracy: 0.8024
Epoch 3/20
990/990 [==============================] - 85s 86ms/step - loss: 0.2894 - accuracy: 0.8899 - val_loss: 0.2881 - val_accuracy: 0.8911
Epoch 4/20
990/990 [==============================] - 85s 86ms/step - loss: 0.2017 - accuracy: 0.9182 - val_loss: 0.2478 - val_accuracy: 0.8992
Epoch 5/20
990/990 [==============================] - 89s 90ms/step - loss: 0.1566 - accuracy: 0.9333 - val_loss: 0.2088 - val_accuracy: 0.9476
Epoch 6/20
990/990 [==============================] - 87s 88ms/step - loss: 0.1088 - accuracy: 0.9636 - val_loss: 0.1647 - val_accuracy: 0.9637
Epoch 7/20
990/990 [==============================] - 85s 85ms/step - loss: 0.0882 - accur

In [8]:
print(model.evaluate(X_test,y_test))

138/138 [==============================] - 2s 16ms/step
[0.25565232902063406, 0.9637681245803833]


In [12]:
from keras.models import load_model

In [13]:
newmodel = load_model('model-006.model')

In [18]:


facesclassi=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

video=cv2.VideoCapture(0)

labels_dict={0:'...NO MASK...',1:'  ...MASK...   '}
color_dict={0:(0,0,255),1:(0,255,0)}
while(True):

    ret,img=video.read()
    img = cv2.flip(img,1,1)
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=facesclassi.detectMultiScale(gray,1.3,5) 

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=newmodel.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('webcam',img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break
        
cv2.destroyAllWindows()
video.release()